In [ ]:
#测试环境Python 3.9.17
!pip install scikit-learn==1.1.2   joblib==1.3.2    zhipuai==1.0.7 gradio==3.41.2

In [ ]:
import gradio as gr
import joblib
import zhipuai

# 设置api_key
zhipuai.api_key = "40626c22a64b429a199b435c3ff0fdc6.suMoOFmzR9KTdEj1"

def chat_with_glm(prompt_input):
    # 和大模型交互
    response = zhipuai.model_api.invoke(
        model="chatglm_std",
        prompt=[#可以自定义模型角色，认知
            {"content": "你好", "role": "user"},
            {"content": "我是一个资深的mbti性格分析师", "role": "assistant"},
            {"content": "你叫什么名字", "role": "user"},
            {"content": "我叫chatGLM", "role": "assistant"},
            {"content": prompt_input, "role": "user"}
        ]
    )
    # 返回大模型的回复
    return response["data"]["choices"][0]["content"]

gr.close_all()
# 加载保存的对象
model = joblib.load('model.pkl')
vectorizer = joblib.load('vectorizer.pkl')
label_encoder = joblib.load('label_encoder.pkl')

def mbti_predict(x, y, z, w):
    combined_text = f"{x}. {y}. {z}. {w}"
    sample_vectorized = vectorizer.transform([combined_text])
    prediction = model.predict(sample_vectorized)
    label = label_encoder.inverse_transform(prediction)[0]
    image_path = "images/{}.png".format(prediction[0])
    mbti_prompt="我的mbti性格类别是"+label+"您建议我在日常生活中需要注意些什么？"#我希望拿到预测标签的对应的mbti类型，然后组成成promp输入给大模型，
    glm_response = chat_with_glm(mbti_prompt)  # 调用上面的方法与大模型交互
    # 将字符串中的 \n 替换为真正的换行
    formatted_response = glm_response.replace("\\n", "\n")
    return image_path, formatted_response

# 定义Gradio界面
interface = gr.Interface(
    fn=mbti_predict,
    inputs=[
        gr.Textbox(lines=1, placeholder="e.g. 我在社交场合中感到很舒适，喜欢与人交往。｜我感到有些紧张，更倾向于观察。", label="你在社交场合中通常感到如何？"),
        gr.Textbox(lines=1, placeholder="e.g. 我基于事实和细节来做决策。｜我依赖直觉和大局观来做决策。", label="当面对决策时，你更倾向于如何考虑？"),
        gr.Textbox(lines=1, placeholder="e.g. 我喜欢有计划并且坚持日常计划。｜我不太喜欢计划，更倾向于随性。", label="你如何看待日常规划和组织？"),
        gr.Textbox(lines=1, placeholder="e.g. 我会逻辑地分析事实，试图说服他人。｜我通常会考虑他人的感受，寻求共识。", label="当与他人争论时，你更倾向于如何反应？"),
        # gr.Textbox(lines=1, placeholder="这里可以咨询大模型，与大模型进行聊天", label="与大模型交互")
    ],
    outputs=[
        gr.Image(type="pil", label="MBTI性格类型"),
        gr.Textbox(readonly=True, label="ChatGLM大语言模型针对你性格的诊断和建议")
    ],
    title="MBTI性格测试系统",
    description="回答以下四个问题来获取您的MBTI性格类型，同时获得大模型给出的诊断建议",
    allow_flagging='never',
)

interface.launch(debug=True, server_name="0.0.0.0", server_port=7865, share=True)


/var/folders/k6/_szdl4rn463c4j047zyfxsh00000gn/T/ipykernel_24752/831301133.py:53: GradioUnusedKwargWarning: You have unused kwarg parameters in Textbox, please remove them: {'readonly': True}
  gr.Textbox(readonly=True, label="ChatGLM大语言模型针对你性格的诊断和建议")


Running on local URL:  http://0.0.0.0:7865
Running on public URL: https://1450ec174e9f27ab9f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
